In [1]:
import whisper
from autocorrect import Speller
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymorphy2
import os
import json
from sklearn.model_selection import train_test_split

morph = pymorphy2.MorphAnalyzer()
spell = Speller('ru')

def lemmatize(text):
    words = text.split()
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
    return ' '.join(res)

# folder path
dir_path_Fraud = r'/root/hackathon-service-template/train-samples/Fraud'
dir_path_NotFraud = r'/root/hackathon-service-template/train-samples/NotFraud'



# list to store files
fraud_files = []
notfraud_files = []


# Iterate directory
for path in os.listdir(dir_path_Fraud):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path_Fraud, path)):
        fraud_files.append(path)
print("Fraud: ", fraud_files)

# Iterate directory
for path in os.listdir(dir_path_NotFraud):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path_NotFraud, path)):
        notfraud_files.append(path)
#         notfraud_files.append(dir_path_NotFraud+'/'+path)
print("Not Fraud: ", notfraud_files)

/usr/local/lib/python3.10/dist-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):


Fraud:  ['out_a_3.wav', 'Nout_f_4.wav', 'out_a_2.wav', 'out_a_1.wav', 'mvd.wav']
Not Fraud:  ['Nout_a_1.wav', 'Nout_a_10.wav', 'Nout_a_9.wav', 'Nout_a_4.wav', 'Nout_a_2.wav']


In [2]:
dir_path_NotFraud = r'/root/hackathon-service-template/text/NotFraud'
dir_path_Fraud = r'/root/hackathon-service-template/text/Fraud'

ok_notfraud_files = []
ok_fraud_files = []

# Iterate directory
for path in os.listdir(dir_path_NotFraud):
    ok_notfraud_files.append(path)
for path in os.listdir(dir_path_Fraud):
    ok_fraud_files.append(path)

ok_fraud_files = [i[:-5] for i in ok_fraud_files]
ok_fraud_files
ok_notfraud_files = [i[:-5] for i in ok_notfraud_files]
ok_notfraud_files

['Nout_a_1.wav',
 'Nout_a_10.wav',
 'Nout_a_4.wav',
 'Nout_a_2.wav',
 'Nout_a_9.wav']

In [36]:
large_model = whisper.load_model("large")

In [5]:
# res_large = large_model.transcribe("/root/hackathon-service-template/train-samples/Fraud/mvd.wav")

In [6]:
# res_large['text']

In [7]:
# text = res_large['text']
# print(spell(text))

In [8]:
# lemmatize(spell(text))

In [37]:
dir_path_Fraud = r'/root/hackathon-service-template/train-samples/Fraud'
dir_path_NotFraud = r'/root/hackathon-service-template/train-samples/NotFraud'

count = 0
for file in fraud_files:
    if file not in ok_fraud_files:
        count+=1
        res_large = large_model.transcribe(dir_path_Fraud+'/'+file)
        with open(f'/root/hackathon-service-template/text/Fraud/{file}.json', 'w') as fp:
            json.dump(res_large['text'], fp)
        print(f"Расшифровано: {count}/{len(fraud_files)} fraud файлов")

count = 0
for file in notfraud_files:
    if file not in ok_notfraud_files:
        count+=1
        res_large = large_model.transcribe(dir_path_NotFraud+'/'+file)
        with open(f'/root/hackathon-service-template/text/NotFraud/{file}.json', 'w') as fp:
                # Преобразование объектов Python в данные 
                # JSON формата, а так же запись в файл 'data.json'
            json.dump(res_large['text'], fp)
        print(f"Расшифровано: {count}/{len(notfraud_files)} notfraud файлов")

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Расшифровано: 1/9 fraud файлов


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Расшифровано: 2/9 fraud файлов


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Расшифровано: 3/9 fraud файлов


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Расшифровано: 4/9 fraud файлов


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score

In [11]:
df = pd.read_csv('data_2.csv')
df.set_index(df.columns[0], inplace=True)

In [12]:
df

,label,content
Unnamed: 0,,
0,1,"hello, i m bank manager of SBI, ur debit card ..."
1,1,Todays Vodafone numbers ending with 4882 are s...
2,0,Please don't say like that. Hi hi hi
3,0,Thank you!
4,0,Oh that was a forwarded message. I thought you...
...,...,...
5919,1,to get 1000 INR voucher please call on 8898655...
5920,1,to get free access of google cloud account hit...
5921,1,to get free AWS cloud account hit on given mes...


In [13]:
X = df['content']
y = df['label']

In [14]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)

In [4]:
vectorizer = CountVectorizer(min_df=0.0005, max_df=0.1)
vec_data_train = vectorizer.fit_transform(X_train).toarray()

NameError: name 'X_train' is not defined

In [16]:
vec_data_val = vectorizer.transform(X_valid).toarray()

In [17]:
multinomial_nb = MultinomialNB()
multinomial_nb.fit(vec_data_train, y_train)

MultinomialNB()

In [18]:
bernoulli_nb = BernoulliNB()
bernoulli_nb.fit(vec_data_train, y_train)

BernoulliNB()

In [19]:
predictions_multi = multinomial_nb.predict(vec_data_val)

In [20]:
predictions_bern = bernoulli_nb.predict(vec_data_val)

In [21]:
accuracy_score(predictions_bern, y_valid)

0.9785166240409208

In [22]:
accuracy_score(predictions_multi, y_valid)

0.9826086956521739

# Обработка

In [4]:
dir_path_NotFraud = r'/root/hackathon-service-template/text/NotFraud'
notfraud_text = []
# Iterate directory
for path in os.listdir(dir_path_NotFraud):
    # check if current path is a file
#     if os.path.isfile(os.path.join(dir_path_NotFraud, path)):
#         notfraud_text.append(path)
    with open(f'/root/hackathon-service-template/text/NotFraud/{path}', 'r') as r:
            # Преобразование объектов Python в данные 
            # JSON формата, а так же запись в файл 'data.json'
        text = json.load(r)
    notfraud_text.append(text)

dir_path_Fraud = r'/root/hackathon-service-template/text/Fraud'
fraud_text = []
# Iterate directory
for path in os.listdir(dir_path_Fraud):
    # check if current path is a file
#     if os.path.isfile(os.path.join(dir_path_NotFraud, path)):
#         notfraud_text.append(path)
    with open(f'/root/hackathon-service-template/text/Fraud/{path}', 'r') as r:
            # Преобразование объектов Python в данные 
            # JSON формата, а так же запись в файл 'data.json'
        text = json.load(r)
    fraud_text.append(text)
nft = pd.DataFrame(notfraud_text)
nft.columns = ['text']
nft.loc[:, 'is_fraud'] = 0

ft = pd.DataFrame(fraud_text)
ft.columns = ['text']
ft.loc[:, 'is_fraud'] = 1

sdf = pd.concat((ft, nft))
sdf.reset_index(inplace=True, drop=True)

In [5]:
sdf

,text,is_fraud
0,Алло. Здравствуйте. Здравствуйте. В газете по...,1
1,"Ефим Владимирович, здравствуйте! Здравствуйте...",1
2,какие действия какие действия на данный момен...,1
3,А это кто? Значит вас беспокоит старший следо...,1
4,"Если вы подтверждаете данную операцию, скажит...",1
5,"Да. Алло. Алло, здравствуйте. Здравствуйте, с...",0
6,"Алло. Алло. Алло, Татьяна. Да. День добрый. Д...",0
7,"Алло, я вас слушаю. Отстанавливаю по суду на ...",0
8,"Здравствуйте, Михаилыч. Да, здравствуйте. Мен...",0
9,"Алло. Алло, день добрый. Добрый. Меня зовут Б...",0


In [6]:
sdf.loc[:, "norm"] = sdf.apply(lambda x: lemmatize(spell(x.text)), axis=1)

In [94]:
sdf

,text,is_fraud,norm
0,"Ваши деньги в банке, вы забираете и передвиже...",1,"ваш деньга в банке, вы забирать и передвижетес..."
1,Алло. Здравствуйте. Здравствуйте. В газете по...,1,алло. здравствуйте. здравствуйте. в газета пок...
2,"Алло. Алло, Анастасия, ВТБ банк с вами связыв...",1,"алло. алло, анастасия, втб банк с вы связывает..."
3,"Алло? Алло? Михаил Юрьевич, здравствуйте. Здр...",1,"алло? алло? михаил юрьевич, здравствуйте. здра..."
4,"Ефим Владимирович, здравствуйте! Здравствуйте...",1,"ефим владимирович, здравствуйте! здравствуйте!..."
5,"Власти Валерьевны, здравствуйте. ВТБ банки На...",1,"власть валерьевны, здравствуйте. втб банк надз..."
6,какие действия какие действия на данный момен...,1,какой действие какой действие на данный момент...
7,А это кто? Значит вас беспокоит старший следо...,1,а это кто? значит вы беспокоить старший следов...
8,"Если вы подтверждаете данную операцию, скажит...",1,"если вы подтверждать данный операцию, сказать ..."
9,"Да. Алло. Алло, здравствуйте. Здравствуйте, с...",0,"да. алло. алло, здравствуйте. здравствуйте, сл..."


In [7]:
X_train = sdf.norm
y_train = sdf.is_fraud

In [8]:
vectorizer = CountVectorizer(min_df=0.0005, max_df=0.1)
vec_data_train = vectorizer.fit_transform(X_train).toarray()

In [9]:
multinomial_nb = MultinomialNB()
multinomial_nb.fit(vec_data_train, y_train)

MultinomialNB()

In [11]:
from joblib import dump, load
dump(multinomial_nb, 'mnb.joblib') 

['mnb.joblib']

In [9]:
import pickle
# with open('cleaned_model', 'wb') as to_write:
#     pickle.dump(multinomial_nb, to_write)

In [10]:
with open('cleaned_model', 'rb') as handle:
    saved_model = pickle.load(handle)

In [11]:
accuracy_score(saved_model.predict(vec_data_train), y_train)

1.0

In [23]:
from skops.io import dump, load

with open("mnb.skops", mode='wb') as w:
    obj = sio.dump(multinomial_nb, w)

In [45]:
accuracy_score(multinomial_nb.predict(vec_data_train), y_train)

1.0

In [30]:
# res_large = large_model.transcribe("/root/hackathon-service-template/train-samples/Fraud/mvd.wav")

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


KeyboardInterrupt: 

In [ ]:
# text = res_large['text']

In [ ]:
# text_df = pd.DataFrame([text])

In [ ]:
# text_df.columns = ['text']

In [ ]:
# text_df.loc[:, "norm"] = text_df.apply(lambda x: lemmatize(spell(x.text)), axis=1)

In [ ]:
# text_df

In [ ]:
# X_train = text_df.norm

In [ ]:
# vectorizer

In [ ]:
# vec_data_val = vectorizer.transform(X_train).toarray()

In [ ]:
# multinomial_nb.predict(vec_data_val)

In [ ]:
# X_train

In [95]:
extra_df = pd.read_csv('additional_data_450.csv')
extra_df.drop(['Unnamed: 0'], axis=1, inplace=True)
extra_df
extra_df

,is_fraud,text,norm
0,1,"здравствуйте, я менеджер банка SBI, срок дейст...","здравствуйте, я менеджер банк sbi, срок действ..."
1,1,"Номера Vodafone, заканчивающиеся на 4882, выби...","номер vodafone, заканчиваться на 4882, выбират..."
2,0,"Пожалуйста, не говори так. Привет, привет","пожалуйста, не говорить так. привет, привет"
3,0,Спасибо!,спасибо!
4,0,"Это было переадресованное послание. Я думала, ...","это быть переадресовать послание. я думала, ты..."
...,...,...,...
445,0,"Они не кладут эти вещи на дороги, чтобы они не...","они не класть этот вещь на дороги, чтобы они н..."
446,0,Когда вы собираетесь ехать на велосипеде?,когда вы собираться ехать на велосипеде?
447,0,"Да, не нужно. Я подожду, пока дождь прекратится.","да, не нужно. я подожду, пока дождь прекратится."
448,0,Есть много компаний. Скажи мне язык.,есть много компаний. сказать я язык.


In [119]:
# new_extra_df
# extra_df[extra_df.is_fraud==0].index
extra_notfraud = np.random.RandomState(42).choice(extra_df[extra_df.is_fraud==0].index, 9, 0)
extra_fraud = np.random.RandomState(42).choice(extra_df[extra_df.is_fraud==1].index, 5, 0)

In [123]:
new_extra_df = pd.concat([extra_df.loc[extra_fraud, :], extra_df.loc[extra_notfraud, :]])

In [124]:
bdf = pd.concat((new_extra_df, sdf))

In [125]:
bdf.reset_index(inplace=True, drop=True)
bdf

,is_fraud,text,norm
0,1,четыре месяца половина цены Оранжевая линия пр...,четыре месяц половина цена оранжевый линия про...
1,1,"здравствуйте, я менеджер банка SBI, срок дейст...","здравствуйте, я менеджер банк sbi, срок действ..."
2,1,"Привет, я очень возбужден, хочу поболтать или ...","привет, я очень возбужден, хотеть поболтать ил..."
3,1,"здравствуйте, сэр, это ваш менеджер банка uniu...","здравствуйте, сэр, это ваш менеджер банк uniun..."
4,1,"здравствуйте, сэр, я из службы поддержки клиен...","здравствуйте, сэр, я из служба поддержка клиен..."
5,0,"Ммм, так вкусно, детка ... милая встряска для ...","ммм, так вкусно, детка ... милый встряска для ..."
6,0,"Извините, я позвоню позже на встрече.","извините, я позвонить поздний на встрече."
7,0,"Вы не против, если я спрошу, что случилось? Вы...","вы не против, если я спрошу, что случилось? вы..."
8,0,Вы всегда вкладываете туда свой бизнес. Ты вык...,вы всегда вкладывать туда свой бизнес. ты выкл...
9,0,"Да, я начал посылать запросы, чтобы сделать эт...","да, я начать посылать запросы, чтобы сделать э..."


In [13]:
X = bdf.norm
y = bdf.is_fraud

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

In [15]:
vectorizer = CountVectorizer(min_df=0.0005, max_df=0.1)
vec_data_train = vectorizer.fit_transform(X_train).toarray()

In [16]:
vec_data_val = vectorizer.transform(X_val).toarray()

In [23]:
multinomial_nb = MultinomialNB()
# multinomial_nb.fit(vec_data_train, y_train)

In [28]:
accuracy_score(multinomial_nb.predict(vec_data_val), y_val)

0.9473684210526315

FOREST

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [22]:
forest = RandomForestClassifier(max_depth=)
forest.fit(vec_data_train, y_train)

RandomForestClassifier(max_depth=5)

In [23]:
accuracy_score(forest.predict(vec_data_val), y_val)

0.8223684210526315

Bern

In [25]:
bern_nb = BernoulliNB()
bern_nb.fit(vec_data_train, y_train)

BernoulliNB()

In [26]:
accuracy_score(bern_nb.predict(vec_data_val), y_val)

0.8486842105263158

ADA with multinimoaial nb

In [29]:
from sklearn.ensemble import AdaBoostClassifier

In [32]:
clf = AdaBoostClassifier(n_estimators=20, random_state=0, base_estimator=MultinomialNB())
clf.fit(vec_data_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


AdaBoostClassifier(base_estimator=MultinomialNB(), n_estimators=20,
                   random_state=0)

In [33]:
accuracy_score(clf.predict(vec_data_val), y_val)

0.8947368421052632

In [88]:
extra_df[extra_df.is_fraud == 0].iloc[42, :].text

'Занятия закончились, где вы?'

In [59]:
import psycopg2

conn = psycopg2.connect(host='localhost',
database='postgres',
user='postgres', password='12345678', sslmode='require')
cur = conn.cursor()
cur.execute('select * from checker;')
conn.commit()
data = np.array(cur.fetchall())
num_test = data[-1, :][1]
ans = data[-1, :][2]
print(num_test, ans)

6 2


In [57]:
len(data)

1

In [19]:
y_train[:3]

0    1
1    1
2    1
Name: is_fraud, dtype: int64

In [24]:
multinomial_nb.partial_fit(vec_data_train[:7], y_train[:7], classes=(0, 1))

MultinomialNB()

In [25]:
multinomial_nb.partial_fit(vec_data_train[7:], y_train[7:])

MultinomialNB()

In [50]:
dir_path_model = r'/root/hackathon-service-template'
is_model = 0# Iterate directory
for path in os.listdir(dir_path_model):
    if path == 'mnb':
        is_model = 1

In [51]:
is_model

1

In [49]:
import pickle
with open('mnb', 'wb') as to_write:
    pickle.dump(multinomial_nb, to_write)

app.py

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS

from solution import solution
import uuid
import whisper
from autocorrect import Speller
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymorphy2
import os
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score
import pickle

morph = pymorphy2.MorphAnalyzer()
spell = Speller('ru')
large_model = whisper.load_model("small")

def lemmatize(text):
    words = text.split()
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
    return ' '.join(res)



print('hello 1')
dir_path_NotFraud = r'/root/hackathon-service-template/text/NotFraud'
notfraud_text = []
# Iterate directory
for path in os.listdir(dir_path_NotFraud):
    # check if current path is a file
#     if os.path.isfile(os.path.join(dir_path_NotFraud, path)):
#         notfraud_text.append(path)
    with open(f'/root/hackathon-service-template/text/NotFraud/{path}', 'r') as r:
            # Преобразование объектов Python в данные 
            # JSON формата, а так же запись в файл 'data.json'
        text = json.load(r)
    notfraud_text.append(text)

print('hello 2')
dir_path_Fraud = r'/root/hackathon-service-template/text/Fraud'
fraud_text = []
# Iterate directory
for path in os.listdir(dir_path_Fraud):
    # check if current path is a file
#     if os.path.isfile(os.path.join(dir_path_NotFraud, path)):
#         notfraud_text.append(path)
    with open(f'/root/hackathon-service-template/text/Fraud/{path}', 'r') as r:
            # Преобразование объектов Python в данные 
            # JSON формата, а так же запись в файл 'data.json'
        text = json.load(r)
    fraud_text.append(text)

print('hello 3')
nft = pd.DataFrame(notfraud_text)
nft.columns = ['text']
nft.loc[:, 'is_fraud'] = 0

ft = pd.DataFrame(fraud_text)
ft.columns = ['text']
ft.loc[:, 'is_fraud'] = 1

sdf = pd.concat((ft, nft))
sdf.reset_index(inplace=True, drop=True)
sdf.loc[:, "norm"] = sdf.apply(lambda x: lemmatize(spell(x.text)), axis=1)

# extra_df = pd.read_csv('additional_data_450.csv')
# extra_df.drop(['Unnamed: 0'], axis=1, inplace=True)
# extra_notfraud = np.random.RandomState(42).choice(extra_df[extra_df.is_fraud==0].index, 9, 0)
# extra_fraud = np.random.RandomState(42).choice(extra_df[extra_df.is_fraud==1].index, 5, 0)
# new_extra_df = pd.concat([extra_df.loc[extra_fraud, :], extra_df.loc[extra_notfraud, :]])
# bdf = pd.concat((new_extra_df, sdf))
# bdf.reset_index(inplace=True, drop=True)

X_train = sdf.norm
y_train = sdf.is_fraud
print('hello 4')
vectorizer = CountVectorizer(min_df=0.0005, max_df=0.1)
vec_data_train = vectorizer.fit_transform(X_train).toarray()
multinomial_nb = MultinomialNB()
multinomial_nb.fit(vec_data_train, y_train)
print("Запущено")

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})

ALLOWED_EXTENSIONS = set(["wav"])


def allowed_file(filename):
    return "." in filename and filename.rsplit(".", 1)[1].lower() in ALLOWED_EXTENSIONS


@app.route("/test-sample", methods=["POST"])
def test_sample():
    # check if the post request has the file part
    if "sample" not in request.files:
        response = jsonify({"message": "NO sample file in the request"})
        response.status = 400
        return response

    file = request.files["sample"]
    uid = uuid.uuid4()
    file.save(str(uid))
    file_path = str(uid)

    if not allowed_file(file.filename):
        response = jsonify({"message": "Only .wav file type is allowed"})
        response.status = 400
        return response

    response = jsonify({"result": solution(large_model, vectorizer, multinomial_nb, file_path)})
    response.status = 200
    return response


if __name__ == "__main__":
    app.run(host='0.0.0.0', port=8080)


solution.py

In [ ]:
from werkzeug.datastructures import FileStorage
import numpy as np
import pymorphy2
from autocorrect import Speller
import pandas as pd
import os


def solution(large_model, vectorizer, multinomial_nb, wav_file) -> int:    
    try:
        morph = pymorphy2.MorphAnalyzer()
        spell = Speller('ru')
        def lemmatize(text):
            words = text.split()
            res = list()
            for word in words:
                p = morph.parse(word)[0]
                res.append(p.normal_form)
            return ' '.join(res)
        res_large = large_model.transcribe(wav_file)
        text = res_large['text']
        text_df = pd.DataFrame([text])
        text_df.columns = ['text']
        text_df.loc[:, "norm"] = text_df.apply(lambda x: lemmatize(spell(x.text)), axis=1)
        X_test = text_df.norm
        vec_data_test = vectorizer.transform(X_test).toarray()
        res = int(multinomial_nb.predict(vec_data_test)[0])
    except:
        res = int(np.random.uniform(low=0.0, high=1.0))
    os.remove(wav_file)
    return res



In [1]:
import numpy as np

In [5]:
np.array(2).reshape(1, ).shape

(1,)